In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/carsdataset/cardata.csv


In [2]:
from sklearn.model_selection import train_test_split

data = pd.read_csv('/kaggle/input/carsdataset/cardata.csv')
data.columns
data.info()
y = data.Kms_Driven
X = data.drop(['Kms_Driven'], axis=1)
X_train,X_val,y_train,y_val =train_test_split(X,y, train_size=0.8, test_size =0.2, random_state=42)

cols_missing = [
    col for col in X_train.columns if X_train[col].isnull().any()]
X_train.drop(cols_missing, axis=1, inplace = True)
X_val.drop(cols_missing, axis =1 , inplace =True)

low_cardinality_cols = [cname for cname in X_train.columns if X_train[cname].nunique()<10 and
                       X_train[cname].dtype=='object']
numerical_cols = [
    cname for cname in X_train.columns if X_train[cname].dtype in ['int64','float64']
]

my_cols = low_cardinality_cols + numerical_cols
X_traino = X_train[my_cols].copy()
X_valid = X_val[my_cols].copy()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301 entries, 0 to 300
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Car_Name       301 non-null    object 
 1   Year           301 non-null    int64  
 2   Selling_Price  301 non-null    float64
 3   Present_Price  301 non-null    float64
 4   Kms_Driven     301 non-null    int64  
 5   Fuel_Type      301 non-null    object 
 6   Seller_Type    301 non-null    object 
 7   Transmission   301 non-null    object 
 8   Owner          301 non-null    int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 21.3+ KB


In [3]:
X_train.head()

,Car_Name,Year,Selling_Price,Present_Price,Fuel_Type,Seller_Type,Transmission,Owner
184,Bajaj Pulsar 150,2008,0.25,0.750,Petrol,Individual,Manual,1
132,Bajaj Avenger 220,2017,0.75,0.950,Petrol,Individual,Manual,0
194,Hero CBZ Xtreme,2008,0.20,0.787,Petrol,Individual,Manual,0
75,etios g,2015,3.95,6.800,Petrol,Dealer,Manual,0
111,Royal Enfield Thunder 350,2016,1.15,1.500,Petrol,Individual,Manual,0


In [4]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute  import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

numerical_transformer = SimpleImputer(strategy='constant')

categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                                          ('onehot', OneHotEncoder(handle_unknown='ignore'))
                                          ])
Preprocessor =ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer,low_cardinality_cols)

    ]
)


In [5]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=50, random_state=0)

In [6]:
from sklearn.metrics import mean_absolute_error

my_pipeline = Pipeline(steps=[('Preprocessor', Preprocessor),
                              ('model',model)])
my_pipeline.fit(X_train, y_train)

preds = my_pipeline.predict(X_val)

score =mean_absolute_error(y_val, preds)
print("MAE: ", score)

MAE:  11902.110710382512


In [7]:
print(preds)
print(y_train)

[ 13516.          44290.04        65240.66        75173.98
  49923.46        44295.54        16934.          32245.6
   6840.          54831.48        13980.52        22490.
  39549.9          4990.          55449.86        33157.92
  18736.          13851.44        12242.           3326.
  21828.          22514.26        62261.72        47922.02
  25037.6         33528.90666667  57160.94        36038.08
   4514.          12564.          25008.          57482.42
  60910.84        67735.8         34604.02        33199.76
  57358.16        76437.6         36793.1         65863.86
 126901.2        119482.72        39812.98        17815.6
  45603.86        35678.          43958.8         72261.38
  41645.22        52670.08        49239.56         3850.
  43521.4         14200.          48343.98        30674.
  25774.6         27473.24        74625.44        18197.58
  31238.        ]
184    26000
132     3500
194    50000
75     36000
111     8700
       ...  
188    18000
71     45000
106